# ICE 1 - Using MongoDB in Julia

**Author**: Nicolas Dussaillant

## Preparation

For this code, the computer specifications were the following:
- Processor: Apple Silicon M1
- OS: MacOS 11.6 (Big Sur)

Previously installed software used here for installations and usage:
- [Homebrew](https://docs.brew.sh/)
- [Anaconda](https://docs.anaconda.com/anaconda/install/mac-os/)

### Installing and preparing Julia:

1. **Download and install Julia:**
    
    For native operation of Julia it is necessary to install v1.7 which is not available through Homebrew, so download and install version 1.7.1 for Apple M-Series [here](https://julialang.org/downloads/) (experimental version).


2. **Create a link to execute Julia from the Terminal:**
    
    Run the following link in the terminal:

    `sudo ln -s /Applications/Julia-1.7.app/Contents/Resources/julia/bin/julia /usr/local/bin/julia`

    (Make sure that directory `/usr/local/bin` exists or create it before running that command)


3. **Download IJulia to use Jupyter Notebooks:**

    Enter Julia typing `julia` in the Terminal. Then press `]`, type `add IJulia` and hit **enter**. After installation, hit **delete** and exit Julia (running `exit()` or pressing `ctrl + D`).
    
    After that, run Jupyter Notebook from Anaconda Navigator, and create a new Julia notebook.
    
    
### Installing and preparing MongoDB

For MongoDB we will use HomeBrew. All the following is run in the **Terminal**.

1. **Add third-party repositories for MongoDB in Brew.** Run:

    `brew tap mongodb/brew`
    
    
2. **Install mongodb-community running**:

    `brew install mongodb-community`
    
    
3. **To start and stop mongodb, use the following commands (although we will run them in this Notebook as well)**:

    `brew services start mongodb-community`

    `brew services stop mongodb-community`
    
    
**Note**: To edit configurations of MongoDB, edit the following file: `/opt/homebrew/etc/mongod.conf`. By default, MongoDB will run on 127.0.0.1:27017


### Install MongoDB package for Julia

We will be using [this package](https://juliapackages.com/p/mongoc) available to use MongoDB in Julia. To install it, do the same as for IJulia, but instead of `add IJulia`, type `add Mongoc`.

***

## Running Julia + MongoDB

#### Try Julia:

In [1]:
println("Hello, world!")

Hello, world!


Run commands in the Terminal using Julia:

In [2]:
run(`echo "Hello, world from the Shell!"`)

Hello, world from the Shell!


Process(`echo 'Hello, world from the Shell!'`, ProcessExited(0))

#### Start MongoDB
(if you start MongoDB, don't forget to stop it when you're done, otherwise it will keep running locally in your computer)

In [3]:
run(`brew services start mongodb/brew/mongodb-community`)

==> Successfully started `mongodb-community` (label: homebrew.mxcl.mongodb-community)


Process(`brew services start mongodb/brew/mongodb-community`, ProcessExited(0))

#### Connect Julia to MongoDB

In [4]:
import Mongoc
mongo_client = Mongoc.Client("localhost", 27017)

Client(URI("mongodb://localhost:27017"))

**Create (or open) the new database "testing-db"**

In [5]:
db = mongo_client["testing-db"]

Database(Client(URI("mongodb://localhost:27017")), "testing-db")

**Populate database with a collection of persons and perform CRUD operations**

In [6]:
# Create collection
persons = db["persons"]

Collection(Database(Client(URI("mongodb://localhost:27017")), "testing-db"), "persons")

**INSERT (C)**

In [7]:
# Create document
person_1 = Mongoc.BSON()

# Fill data in the document
person_1["first-name"] = "Ariana"
person_1["last-name"] = "Grande"
person_1["occupation"] = "artist"
person_1["gender"] = "female"

# Insert into collection
push!(persons, person_1)

# Repeat for persons 2 and 3 with different methods
# Same line Dict
person_2 = Mongoc.BSON( "first-name" => "Elon",
                        "last-name"  => "Musk",
                        "occupation" => "CEO",
                        "gender"     => "male")

# With JSON string
person_3 = Mongoc.BSON("""
                        {
                        "first-name": "Will",
                        "last-name" : "Smith",
                        "occupation": "actor",
                        "gender"    : "male"
                        }
                        """)

# Add persons 2 and 3 together
append!(persons, [ person_2, person_3 ])


Mongoc.BulkOperationResult{Union{Nothing, Mongoc.BSONObjectId}}(BSON("{ "nInserted" : 2, "nMatched" : 0, "nModified" : 0, "nRemoved" : 0, "nUpserted" : 0, "writeErrors" : [  ] }"), 0x00000001, Union{Nothing, Mongoc.BSONObjectId}[BSONObjectId("61fce3ec0a9e54f3ff3e9923"), BSONObjectId("61fce3ec0a9e54f3ff3e9924")])

In [8]:
# Print what is in the collection
for person in persons
    println("$(person["first-name"]) $(person["last-name"])'s work is $(person["occupation"])")
end

Ariana Grande's work is artist
Elon Musk's work is CEO
Will Smith's work is actor


**SELECT (R)**

In [9]:
# Select One
selector = Mongoc.BSON("gender" => "male")
p = Mongoc.find_one(persons, selector)
println(p)

BSON("{ "_id" : { "$oid" : "61fce3ec0a9e54f3ff3e9923" }, "first-name" : "Elon", "last-name" : "Musk", "occupation" : "CEO", "gender" : "male" }")


In [10]:
# Select many
selector = Mongoc.BSON("gender" => "male")
pp = Mongoc.find(persons, selector)
for i in pp
    println(i)
end

BSON("{ "_id" : { "$oid" : "61fce3ec0a9e54f3ff3e9923" }, "first-name" : "Elon", "last-name" : "Musk", "occupation" : "CEO", "gender" : "male" }")
BSON("{ "_id" : { "$oid" : "61fce3ec0a9e54f3ff3e9924" }, "first-name" : "Will", "last-name" : "Smith", "occupation" : "actor", "gender" : "male" }")


**UPDATE (U)**

In [11]:
selector = Mongoc.BSON("gender" => "male")
update = Mongoc.BSON("\$set" => Dict("language" => "en-us"))
Mongoc.update_many(persons, selector, update)

Mongoc.BSON with 3 entries:
  "modifiedCount" => 2
  "matchedCount"  => 2
  "upsertedCount" => 0

In [12]:
for person in persons
    println(person)
end

BSON("{ "_id" : { "$oid" : "61fce3eb0a9e54f3ff3e9922" }, "first-name" : "Ariana", "last-name" : "Grande", "occupation" : "artist", "gender" : "female" }")
BSON("{ "_id" : { "$oid" : "61fce3ec0a9e54f3ff3e9923" }, "first-name" : "Elon", "last-name" : "Musk", "occupation" : "CEO", "gender" : "male", "language" : "en-us" }")
BSON("{ "_id" : { "$oid" : "61fce3ec0a9e54f3ff3e9924" }, "first-name" : "Will", "last-name" : "Smith", "occupation" : "actor", "gender" : "male", "language" : "en-us" }")


**DELETE (D)**

In [13]:
selector = Mongoc.BSON("first-name" => "Elon")
Mongoc.delete_many(persons, selector)

Mongoc.BSON with 1 entry:
  "deletedCount" => 1

In [14]:
for person in persons
    println(person)
end

BSON("{ "_id" : { "$oid" : "61fce3eb0a9e54f3ff3e9922" }, "first-name" : "Ariana", "last-name" : "Grande", "occupation" : "artist", "gender" : "female" }")
BSON("{ "_id" : { "$oid" : "61fce3ec0a9e54f3ff3e9924" }, "first-name" : "Will", "last-name" : "Smith", "occupation" : "actor", "gender" : "male", "language" : "en-us" }")


In [15]:
# Delete all
selector = Mongoc.BSON()
Mongoc.delete_many(persons, selector)

Mongoc.BSON with 1 entry:
  "deletedCount" => 2

In [16]:
length(persons)

0

***

## Ending: Delete and close everything

In [17]:
# Delete collection
Mongoc.drop(persons)

# Delete database
Mongoc.drop(db)

# Finish MongoDB
run(`brew services stop mongodb/brew/mongodb-community`)

Stopping `mongodb-community`... (might take a while)
==> Successfully stopped `mongodb-community` (label: homebrew.mxcl.mongodb-community)


Process(`brew services stop mongodb/brew/mongodb-community`, ProcessExited(0))